In [4]:
import sys
import os

sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))

In [5]:
import pandas as pd 
import numpy as np 
import pyodbc
import ast
from datetime import datetime
import json
import re
import AccConv as ac

In [6]:
ac.SupressWarnings()

In [7]:
def string_to_list(s):
    try:
        result = ast.literal_eval(s)
        if isinstance(result, list):
            return result
        else:
            raise ValueError("The string does not represent a list.")
    except Exception as e:
        print(f"Error: {e}")
        return None


In [8]:
def calculate_bill(
    daily_demand_df,
    usage_kwh,
    start_date,
    end_date,
    rate_data,
    contract_demand_kW=0.0
):
    """
    Calculate a detailed electric delivery bill based on as-used demand and energy usage.

    Parameters:
    - daily_demand_df (DataFrame): Must contain 'Date', 'MidPeakDemand_kW', 'PeakDemand_kW'
    - usage_kwh (float): Total energy usage during billing period
    - start_date (str or datetime): Billing period start date
    - end_date (str or datetime): Billing period end date
    - rate_data (dict): Rate structure extracted via extract_rate_data
    - contract_demand_kW (float): Contracted demand in kW

    Returns:
    - dict: Itemized breakdown of charges and total bill
    """

    # --- Date Handling ---
    start_date = pd.to_datetime(start_date)
    end_date = pd.to_datetime(end_date)
    daily_demand_df['Date'] = pd.to_datetime(daily_demand_df['Date'])

    # --- Weekday Filtering for Billing Period ---
    billing_df = daily_demand_df[
        (daily_demand_df['Date'] >= start_date) &
        (daily_demand_df['Date'] <= end_date) &
        (daily_demand_df['Date'].dt.weekday < 5)
    ].copy()

    # --- Seasonal Masks ---
    billing_df['Month'] = billing_df['Date'].dt.month
    summer_mask = billing_df['Month'].isin([6, 7, 8, 9])  # June through Sept

    # --- Contract Demand Charge ---
    contract_demand_rate = rate_data.get('contract_demand_rate', 0.0)
    contract_demand_charge = contract_demand_kW * contract_demand_rate

    # --- Summer As-Used Demand ---
    summer_df = billing_df[summer_mask]
    midpeak_kW_summer = summer_df['MidPeakDemand_kW'].fillna(0).sum()
    peak_kW_summer = summer_df['PeakDemand_kW'].fillna(0).sum()

    midpeak_rate = rate_data.get('midpeak_rate_summer', 0.0)
    peak_rate = rate_data.get('peak_rate_summer', 0.0)
    demand_charge_summer = (
        midpeak_kW_summer * midpeak_rate +
        peak_kW_summer * peak_rate
    )

    # --- Non-Summer As-Used Demand ---
    nonsummer_df = billing_df[~summer_mask][['MidPeakDemand_kW', 'PeakDemand_kW']].fillna(0)
    # Use highest daily kW of either midpeak or peak per day
    nonsummer_daily_max = nonsummer_df.max(axis=1).sum()

    as_used_rate_nonsummer = rate_data.get('as_used_rate_nonsummer', 0.0)
    demand_charge_nonsummer = nonsummer_daily_max * as_used_rate_nonsummer

    # --- Energy-Based Surcharge ---
    surcharge_rate = rate_data.get('surcharge_rate', 0.0)
    surcharge_charge = usage_kwh * surcharge_rate
    kwh_breakdown = rate_data.get('kwh_charge_breakdown', {})
    energy_breakdown = {
        k: {
            'rate_per_kWh': v,
            'charge': round(v * usage_kwh, 10)
        } for k, v in kwh_breakdown.items()
    }


    # --- Fixed Monthly Charges ---
    customer_charge = rate_data.get('customer_charge', 0.0)
    processing_charge = rate_data.get('processing_charge', 0.0)
    fixed_charges = customer_charge + processing_charge

    # --- Total Delivery Bill ---
    total = (
        fixed_charges +
        contract_demand_charge +
        demand_charge_summer +
        demand_charge_nonsummer +
        surcharge_charge
    )

    # --- Return Detailed Breakdown ---
    return {
        'customer_charge': {
            'amount': round(customer_charge, 2),
            'description': 'Fixed monthly customer service charge'
        },
        'processing_charge': {
            'amount': round(processing_charge, 2),
            'description': 'Billing & payment processing fee'
        },
        'energy_surcharge': {
                'amount': round(surcharge_charge, 2),
                'rate_per_kWh': round(surcharge_rate, 10),
                'usage_kWh': usage_kwh,
                'breakdown': energy_breakdown,
                'description': 'Total of all $/kWh delivery surcharges'
            },

        'demand_charge_summer': {
            'amount': round(demand_charge_summer, 2),
            'midpeak_kWh_sum': round(midpeak_kW_summer, 2),
            'midpeak_rate': midpeak_rate,
            'peak_kWh_sum': round(peak_kW_summer, 2),
            'peak_rate': peak_rate,
            'description': 'Sum of midpeak and peak weekday demand (Summer)'
        },
        'demand_charge_nonsummer': {
            'amount': round(demand_charge_nonsummer, 2),
            'sum_daily_max_kW': round(nonsummer_daily_max, 2),
            'rate': as_used_rate_nonsummer,
            'description': 'Sum of daily weekday max(kW) for Non-Summer'
        },
        'contract_demand_charge': {
            'amount': round(contract_demand_charge, 2),
            'kW': contract_demand_kW,
            'rate': contract_demand_rate,
            'description': 'Fixed monthly rate based on contracted demand'
        },
        'total': round(total, 2)
    }


In [9]:
def extract_rate_data(rates, charges_df, start_date, end_date, city="New York City"):
    if isinstance(start_date, str): start_date = pd.to_datetime(start_date)
    if isinstance(end_date, str): end_date = pd.to_datetime(end_date)
    billing_days = (end_date - start_date).days

    def get_effective_rate(entries, field, date_col='EffectiveDate'):
        df = pd.DataFrame(entries)
        df[date_col] = pd.to_datetime(df[date_col])
        df = df[df[date_col] <= end_date]
        if df.empty:
            return 0.0
        return float(df.sort_values(date_col).iloc[-1][field])

    def extract_summer_demand_rate(entries, start_time, end_time):
        df = pd.DataFrame(entries)
        df['EffectiveDate'] = pd.to_datetime(df['EffectiveDate'])
        df = df[
            (df['StartTime'] == start_time) &
            (df['EndTime'] == end_time) &
            (df['Season'].str.strip().str.lower() == 'june-sept') &
            (df['EffectiveDate'] <= end_date)
        ]
        if df.empty:
            return 0.0
        return float(df.sort_values('EffectiveDate').iloc[-1]['RatekW'])

    def get_weighted_average(entries, field):
        df = pd.DataFrame(entries)
        df['EffectiveDate'] = pd.to_datetime(df['EffectiveDate'])
        df = df[df['EffectiveDate'] <= end_date]
        df = df.sort_values('EffectiveDate')
        if df.empty:
            return 0.0

        weights, rates = [], []
        for i, row in df.iterrows():
            eff_date = max(row['EffectiveDate'], start_date)
            if i + 1 < len(df):
                next_date = min(end_date, df.iloc[i + 1]['EffectiveDate'] )#- pd.Timedelta(days=1))
            else:
                next_date = end_date
            days = (next_date - eff_date).days #+ 1
            if days < 0:
                continue

            weights.append(days)
            rates.append(float(row[field]))

        total_days = sum(weights)
        return sum(w * r for w, r in zip(weights, rates)) / total_days if total_days > 0 else 0.0

    def extract(table_key, field_name, desc_match, charge_meta):
        entries = [e for e in rates.get(table_key, []) if desc_match.lower() in e['Description'].lower()]
        if not entries:
            print(f'No charge found for {desc_match}')
            return 0.0

        if charge_meta.get('Weighted', False):
            rate = get_weighted_average(entries, field_name)
        else:
            rate = get_effective_rate(entries, field_name)

        if charge_meta.get('Prorated', False):
            rate *= billing_days / 30

        return rate

    rate_data = {}
    kwh_charges = {}

    for _, row in charges_df.iterrows():
        desc = row['Description']
        key = desc.lower().strip()#.replace(" ", "_").replace("-", "").replace("/", "").replace(":", "")

        if row['Unit'] == 'kWh' and row['ServiceTypeId'] in [0, 2]:
            rate = extract('Energy_Table', 'RatekWh', desc, row)
            kwh_charges[key] = round(rate, 10)

        elif row['Unit'] == 'kW':
            rate = extract('Demand_Table', 'RatekW', desc, row)

        elif 'Customer Charge' in desc:
            rate = extract('ServiceCharge_Table', 'Rate', desc, row)

        elif 'Processing' in desc:
            rate = extract('OtherCharges_Table', 'ChargeType', desc, row)

        else:
            continue

        rate_data[key] = round(rate, 10)

    # Ensure Monthly Adjustment Clause is captured if present
    # mac_desc = "Monthly Adjustment Clause"
    # mac_entries = [
    #     e for e in rates.get("Energy_Table", [])
    #     if mac_desc.lower() in e["Description"].lower()
    # ]
    # if mac_entries:
    #     mac_df = pd.DataFrame(mac_entries)
    #     mac_df["EffectiveDate"] = pd.to_datetime(mac_df["EffectiveDate"])
    #     mac_df = mac_df[mac_df["EffectiveDate"] <= end_date]
    #     if not mac_df.empty:
    #         latest_mac = mac_df.sort_values("EffectiveDate").iloc[-1]
    #         kwh_charges['monthly_adjustment_clause'] = round(float(latest_mac["RatekWh"]), 10)

    rate_data_mapped = {
        'midpeak_rate_summer': extract_summer_demand_rate(rates["DemandTime_Table"], "800", "1759"),
        'peak_rate_summer': extract_summer_demand_rate(rates["DemandTime_Table"], "800", "2159"),
        'as_used_rate_nonsummer': extract(
            'DemandTime_Table', 'RatekW',
            'As-used Daily Demand Delivery Charge',
            {'Weighted': False, 'Prorated': False}
        ),
        'surcharge_rate': sum(v for v in kwh_charges.values()),
        'customer_charge': rate_data.get('customer_charge', 0.0),
        'processing_charge': rate_data.get('billing_and_payment_processing_ch', 0.0),
        'contract_demand_rate': extract(
            'Demand_Table', 'RatekW',
            'Contract Demand Delivery Charge',
            {'Weighted': False, 'Prorated': True}
        ),
        'delivery_tax_rate': 0.0,  # not implemented
        'kwh_charge_breakdown': kwh_charges
    }

    return rate_data_mapped

In [10]:
conn1 = pyodbc.connect('Driver={SQL Server};'
                              'Server=UTIL-PROD-DB;'
                              'Database=NewClientInfo;')
sql1 = """SELECT TOP (1000) [Id]
      ,[RateAcuityRateId]
      ,[EffectiveDate]
      ,[RateHistory]
      ,[CreatedBy]
      ,[CreatedDate]
      ,[ModifiedBy]
      ,[ModifiedDate]
  FROM [ExternalData].[dbo].[RateAcuityRateHistory]
  WHERE RateAcuityRateId = 22
"""
history = string_to_list(pd.read_sql(sql1,conn1)['RateHistory'][0])[0]

In [11]:
sql1 = """
SELECT TOP (1000) [Id]
      ,[RateAcuityRateId]
      ,[RateAcuityChargeId]
      ,[RateAcuityChargeDescription] AS 'Description'
      ,[RateAcuityChargeSeason]
      ,[RateAcuityChargeStartDate]
      ,[RateAcuityChargeEndDate]
      ,[RateAcuityChargeStartTime]
      ,[RateAcuityChargeEndTime]
      ,[RateAcuityChargeTimeOfDay]
      ,[RateAcuityChargeDeterminant] AS 'Unit'
      ,[ChargeTypeId]
      ,[ChargeParameterTypeId]
      ,[UsageType]
      ,[WeightedAverage] AS 'Weighted'
      ,[Prorated]
      ,[Block]
      ,[ServiceTypeId]
      ,[RatchetChargeDetailsId]
      ,[Complete]
      ,[CreatedBy]
      ,[CreatedDate]
      ,[ModifiedBy]
      ,[ModifiedDate]
  FROM [ExternalData].[dbo].[ChargeConfiguration]
  WHERE RateAcuityRateId = 22
  AND RateAcuityChargeDescription NOT LIKE '%Average Supply Charge%'
  AND [ChargeTypeId] IN (0, 2)

"""
charges = pd.read_sql(sql1,conn1)

In [12]:
daily_demand_df = pd.read_excel(r"H:\Downloads\Sals Calcs\AsUsedDailyDemand.xlsx")

In [13]:
monthly_adj_desc = "Revenue Decoupling mechanism adjustment"

entries = [e for e in history.get("Energy_Table", []) if monthly_adj_desc.lower() in e["Description"].lower()]
for e in entries:
    print(f"{e['EffectiveDate']} | {e['Description']} | {e.get('RatekWh', 'N/A')}")


2/1/2024 12:00:00 AM | Revenue Decoupling Mechanism Adjustment | -0.0015000
8/1/2024 12:00:00 AM | Revenue Decoupling Mechanism Adjustment | 0.0000300
2/1/2025 12:00:00 AM | Revenue Decoupling Mechanism Adjustment | -0.0023100


In [14]:
set(charges['Description'].unique().tolist())

{'Ancillary Services Charges',
 'Arrears Management Program Recovery Surcharge',
 'As-used Daily Demand Delivery Charge',
 'Billing and Payment Processing Charge, per bill',
 'Capacity Component - Market Value',
 'Capacity Component - Out of Market Value',
 'Clean Energy Fund Surcharge',
 'Clean Energy Standard Delivery Surcharge - Costs Associated with the NYSERDA Residential and Retail',
 'Clean Energy Standard Delivery Surcharge - Costs associated with Backstop Charges',
 'Clean Energy Standard Supply Charge - Renewable Energy and Alternative Compliance',
 'Clean Energy Standard Supply Charge - Zero Emission',
 'Clean Energy Supply Charge - Environmental Component',
 'Community Credit',
 'Contract Demand Delivery Charge',
 'Costs Associated with the NYSERDA Integrated Energy Data Resource Phase 2 Program',
 'Customer Charge',
 'Delivery Revenue Surcharge',
 'Demand Reduction Value (DRV)',
 'Dynamic Load Surcharge',
 'Electric Vehicle Make Ready Surcharge',
 'Energy Efficiency Tracke

In [22]:
history['ServiceCharge_Table']

[{'Pending': '',
  'ScheduleID': '21239',
  'Description': 'Customer Charge',
  'Rate': '66.00000',
  'EffectiveDate': '1/1/2024 12:00:00 AM',
  'ServiceCharge_PKID': '22151',
  'ChargeUnit': 'per month'},
 {'Pending': '',
  'ScheduleID': '21239',
  'Description': 'Customer Charge',
  'Rate': '71.00000',
  'EffectiveDate': '1/1/2025 12:00:00 AM',
  'ServiceCharge_PKID': '22151',
  'ChargeUnit': 'per month'}]

In [20]:
charges[charges['Description'] == 'Customer Charge'].iloc[0]

Id                                                 5207
RateAcuityRateId                                     22
RateAcuityChargeId                                22151
Description                             Customer Charge
RateAcuityChargeSeason                                 
RateAcuityChargeStartDate                              
RateAcuityChargeEndDate                                
RateAcuityChargeStartTime                              
RateAcuityChargeEndTime                                
RateAcuityChargeTimeOfDay                              
Unit                                          per month
ChargeTypeId                                          0
ChargeParameterTypeId                                17
UsageType                                              
Weighted                                           True
Prorated                                           True
Block                                             False
ServiceTypeId                                   

In [16]:
rates = extract_rate_data(history, charges, '12/31/2024', '1/31/2025', city="New York City")

bill = calculate_bill(
    daily_demand_df,
    49515,
    '12/31/2024', '1/31/2025',
    rates,
    contract_demand_kW=472
)
def print_bill_summary(bill):
    print("Itemized Bill Summary:\n")
    
    for key, details in bill.items():
        if isinstance(details, dict):
            print(f"{key.replace('_', ' ').title()}: ${details.get('amount', 0):,.2f}")
            
            # Special handling for energy surcharge breakdown
            if key == "energy_surcharge" and 'breakdown' in details:
                print(f"    - Rate per kWh: {details.get('rate_per_kWh', 0):.6f}")
                print(f"    - Usage: {details.get('usage_kWh', 0):,} kWh")
                print(f"    - Breakdown:")
                for charge, info in details['breakdown'].items():
                    charge_label = charge.replace('_', ' ').title()
                    charge_amt = info.get('charge', 0.0)
                    charge_rate = info.get('rate_per_kWh', 0.0)
                    print(f"        • {charge_label}: ${charge_amt:,.2f} @ {charge_rate:.10f}/kWh")
            else:
                for k, v in details.items():
                    if k != 'amount':
                        print(f"    - {k.replace('_', ' ').capitalize()}: {v}")
        else:
            print(f"{key.replace('_', ' ').title()}: ${details:,.2f}")

print_bill_summary(bill)


No charge found for As-used Daily Demand Delivery Charge
No charge found for As-used Daily Demand Delivery Charge
No charge found for As-used Daily Demand Delivery Charge
Itemized Bill Summary:

Customer Charge: $0.00
    - Description: Fixed monthly customer service charge
Processing Charge: $0.00
    - Description: Billing & payment processing fee
Energy Surcharge: $860.42
    - Rate per kWh: 0.017377
    - Usage: 49,515 kWh
    - Breakdown:
        • Mac Reconciliation: $283.82 @ 0.0057320000/kWh
        • Mac Uncollectible-Bill Expense: $3.22 @ 0.0000650000/kWh
        • Mac Transition Adjustment: $0.30 @ 0.0000060000/kWh
        • Monthly Adjustment Clause: $230.28 @ 0.0046506452/kWh
        • Revenue Decoupling Mechanism Adjustment: $1.49 @ 0.0000300000/kWh
        • Clean Energy Fund Surcharge: $326.80 @ 0.0066000000/kWh
        • Energy Efficiency Tracker: $0.00 @ 0.0000000000/kWh
        • Ancillary Services Charges: $0.00 @ 0.0000000000/kWh
        • Nypa Transmission Adjustm

In [28]:
10615.20 * 1.04

11039.808

In [18]:
rates = extract_rate_data(history, charges, '6/28/2024', '7/30/2024', city="New York City")

bill = calculate_bill(
    daily_demand_df,
    64509,
    '6/28/2024', '7/30/2024',
    rates,
    contract_demand_kW=424
)

print_bill_summary(bill)

No charge found for As-used Daily Demand Delivery Charge
No charge found for As-used Daily Demand Delivery Charge
No charge found for As-used Daily Demand Delivery Charge
Itemized Bill Summary:

Customer Charge: $0.00
    - Description: Fixed monthly customer service charge
Processing Charge: $0.00
    - Description: Billing & payment processing fee
Energy Surcharge: $366.44
    - Rate per kWh: 0.005680
    - Usage: 64,509 kWh
    - Breakdown:
        • Mac Reconciliation: $184.37 @ 0.0028580000/kWh
        • Mac Uncollectible-Bill Expense: $2.52 @ 0.0000390000/kWh
        • Mac Transition Adjustment: $0.45 @ 0.0000070000/kWh
        • Monthly Adjustment Clause: $258.32 @ 0.0040043750/kWh
        • Revenue Decoupling Mechanism Adjustment: $-96.76 @ -0.0015000000/kWh
        • Clean Energy Fund Surcharge: $0.00 @ 0.0000000000/kWh
        • Energy Efficiency Tracker: $0.00 @ 0.0000000000/kWh
        • Ancillary Services Charges: $0.00 @ 0.0000000000/kWh
        • Nypa Transmission Adjust

In [19]:
15285.38 * 1.022099042 * 1.08875

17009.72879220224

In [ ]:
8.66 * (33/30)

9.526000000000002

In [ ]:
total = 16438.75
.08875*total + total

17897.6890625

6/28/24- 7/30/24 – TOTAL delivery charges =  $  17,199.25 


12/31/24 – 1/30/25 – TOTAL delivery charges =  $  11,014.49 


In [ ]:
472 * 8.75

4130.0